# Data Cleaning of Bike Lane Data

San Francisco has a bike system but some argue that there are improvements to be made. Many streets do not have bike lanes, and the bike lanes that do exist do not have lane dividers and or cars allowd to be parked in the lane. 

Opponents of bike lanes say that they do not have an affect on accident outcomes and can away parking spaces that would allow customers to park near businesses.

I would like to study the bike lanes in SF using data from [DataSF](https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry). After using [lane-breach](https://github.com/lanebreach)'s API to match traffic accident data to bike lanes in SF, I would like visualize the accident data, as well as run a regression to attempt to quantify the impact of bike lanes on accidents


[Vision Zero SF](https://sfgov.org/scorecards/transportation/traffic-fatalities) apparently tried to do a study, but
I start with a dataframe of traffic accident police reports. Then I:
    1. Map Accidents to bike lanes using lane breach's API to map accidents to bike lanes
    2. Read Road map of SF, and merge this road map to the traffic accident/bike lane dataframe

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import datetime as dt
import time
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pyplot import Axes
from shapely.geometry import Point, shape
import requests
from itertools import compress
from plotnine import *

#!/usr/bin/python
import psycopg2
import pgdb

ModuleNotFoundError: No module named 'geopandas'

## Accident Data

We are looking at Accident data pulled from DataSF from the years 2003 through 2018. All these accidents are just general bike lane accidents

In [2]:
# pull in police report data
reports_03_18 = pd.read_csv('CodeforSF/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv', engine='python')
reports_03_18[( reports_03_18['Descript'].str.contains("^TRAFFIC") )  & ( reports_03_18['Descript'].str.contains("^TRAFFIC VIOLATION") == False )]

# filter by reports which contain the word traffic, sans traffic violations
traffic_reports = reports_03_18[( reports_03_18['Descript'].str.contains("^TRAFFIC") )  & ( reports_03_18['Descript'].str.contains("^TRAFFIC VIOLATION") == False )]


#traffic_reports_orig_index = reports_03_18[ reports_03_18['Descript'] == "TRAFFIC ACCIDENT" ]


#reports_03_18[ reports_03_18['Descript'] == "TRAFFIC ACCIDENT" ]
traffic_reports = traffic_reports.reset_index(drop = True )

Let's look at unique values to figure out what can we analyze

In [3]:
def unique_values(df):
    id = []
    for i in df:
        try: 
            length_of_column = len(np.unique(df[i]))
            if ( length_of_column > (df.shape[0]*.01)): # if greater than a 1/6 of the dataset shape, don't even bother printing them 
                id.append( i )
            elif length_of_column < (df.shape[0]*.01):
                print(i, df[i].value_counts(),"\n" )
        except: next
    print( "a lot of unique values in these columns: ", id)

In [20]:
unique_values(traffic_reports)

Category NON-CRIMINAL      1238
OTHER OFFENSES     916
Name: Category, dtype: int64 

Descript TRAFFIC ACCIDENT                                 1238
TRAFFIC COLLISION, HIT & RUN, PROPERTY DAMAGE     616
TRAFFIC COLLISION, HIT & RUN, INJURY              300
Name: Descript, dtype: int64 

DayOfWeek Friday       330
Saturday     317
Wednesday    316
Monday       315
Thursday     305
Tuesday      288
Sunday       283
Name: DayOfWeek, dtype: int64 

PdDistrict MISSION       307
SOUTHERN      296
BAYVIEW       286
INGLESIDE     277
TARAVAL       253
NORTHERN      215
CENTRAL       166
RICHMOND      157
PARK          122
TENDERLOIN     75
Name: PdDistrict, dtype: int64 

Resolution NONE                                      1040
ARREST, BOOKED                             893
ARREST, CITED                              150
JUVENILE BOOKED                             15
UNFOUNDED                                   13
EXCEPTIONAL CLEARANCE                       11
COMPLAINANT REFUSES TO PROSECUTE  

Just from eyeballing the sample sizes, given that I will at least be having 15 years of data with bike lanes, I wouldn't trust estimates of coefficients for variables tellings us whether or not there's a bike lane buffer, whether or not there's a bike lane barrier, and whether or not there's a greenwave (light on ground). No bikelanes are in the opposite direction of the bike route. 

At the very least, it would be interesting to do analysis for sharrows and for bike routes/lanes

## Map Accidents to bike lanes using lane breach's API

I'm using lane breach's API to to map coordinates of bike lanes to actual accidents

In [ ]:
URL = "https://lane-breach.herokuapp.com/api/bikeway_networks?"

In [ ]:
# distance is in meters to the bike lane 
# returns a string that is firstly a array with _
# if there is no bike lane, it may be better to just skip it because it would save us subsetting and the work of appending.
def accident_near_bikelane( Y, X ):
    output =[0, '', 'nolane', 0 ]
    PARAMS = {'lat': Y, 'long': X}
    r = requests.get(url = URL, params = PARAMS)
    save = r.text
    if r.text == 'null':
        return([])
    else:
        return(r.text)

In [ ]:
bikelane_x = traffic_reports['X'].reset_index( drop = True)
bikelane_y = traffic_reports['Y'].reset_index( drop = True)

In [ ]:
%%time 

## START HERE
bikelane_exist = []
bikelane_info = []

#for i in range(0, accident_data.shape[0]):
for i in range(0, len(bikelane_x)):
    accident_bike_info = accident_near_bikelane( bikelane_y[i], bikelane_x[i])
    
    bikelane_info.append(accident_bike_info)

    if len(accident_bike_info) == 0:
        bikelane_exist.append(False)
    else:
        bikelane_exist.append(True)
#print(bikelane_exist) # use this to subset
#print(bikelane_info)

#%slack traffic report mapping to bike lane code has finished 

In [ ]:
def removeobjectid( save ):
    # parse out the objectid from output
    try:
        #output = float(save[ (save.find("objectid\"", 0, len(save)) +10):( save.find("qtr", 0, len(save))-2) ])
        #output = float(save[ (save.find("cnn\"", 0, len(save)) +5):( save.find("contra", 0, len(save))-2) ])
        output = save[ (save.find("globalid\"", 0, len(save)) +11):( save.find("greenwave", 0, len(save))-3) ]
    except:
        output = ""
    return(output)

In [ ]:
bikelane_objectid = pd.DataFrame({'globalid': np.array(bikelane_info)})
bikelane_objectid['globalid'] = bikelane_objectid['globalid'].apply( lambda x: removeobjectid(x) )

In [ ]:
def removeinstallyr( save ):
    # parse out the objectid from output
    try:
        output = int(save[ (save.find("install_yr\"", 0, len(save)) +12):( save.find("last_edite", 0, len(save))-2) ])
    except:
        output = int(0)
    return(output)

In [ ]:
bikelane_installyr = pd.DataFrame({'install_yr': np.array(bikelane_info)})
bikelane_installyr['install_yr'] = bikelane_installyr['install_yr'].apply( lambda x: removeinstallyr(x) )

In [423]:
#traffic_reports.columns, bikelane_info2.columns, bikelane_exist2.columns, traffic_reports_bikelaneobjectids.columns

In [424]:
bikelane_exist2= pd.DataFrame( {'bikelane_exist': np.array(bikelane_exist)} )

# merge bike lane info to the traffic accidents by
traffic_reports_bikelaneobjectids = pd.concat([traffic_reports, bikelane_objectid, bikelane_exist2, bikelane_installyr ], axis=1, sort=False)

In [ ]:
# convert date of accident to datetime object. Compare bike lane installation date to it
traffic_reports_bikelaneobjectids['Date'] = pd.to_datetime(traffic_reports['Date'])
traffic_reports_bikelaneobjectids['accident_year'] = traffic_reports['Date'].apply( lambda x: x.year)

This is the final list of traffic accidents mapped to bike lanes object id. Out of ~2000, ~900 had accidents. Due to time it takes for the data frame to load, the traffic bike lane dataset was written to file, and I'm uploading it here



In [ ]:
# save to csv since that script is a pain
traffic_reports_bikelaneobjectids.to_csv('BikewayNetwork_edit/traffic_reports_bikelaneobjectid.csv')

In [ ]:
## Read CSV

In [9]:
traffic_reports_bikelaneobjectids = pd.read_csv('BikewayNetwork_edit/traffic_reports_bikelaneobjectid.csv')
traffic_reports_bikelaneobjectids = traffic_reports_bikelaneobjectids.reset_index(drop = True)

In [10]:
traffic_reports_bikelaneobjectids['Date'] = pd.to_datetime(traffic_reports_bikelaneobjectids['Date'])

Now let's join the traffic dataset to the bike lane dataframe in order to figure out what was made first and then second.

More information about the bike lane dataset can be found [here](https://data.sfgov.org/Transportation/SFMTA-Bikeway-Network/ygmz-vaxd)

In [4]:
bikeway_map3 = gpd.read_file('BikewayNetwork_edit/bikeway_map_wcensus.shp')

In [5]:
unique_values(bikeway_map3)

biap NO     5155
No       14
YES       6
Name: biap, dtype: int64 

buffered NO     4929
YES     246
Name: buffered, dtype: int64 

contraflow NO     5160
No       10
YES       5
Name: contraflow, dtype: int64 

direct 2W    4966
1W     209
Name: direct, dtype: int64 

double 0.0    2705
1.0    2450
Name: double, dtype: int64 

facility_t CLASS III    2808
CLASS II     1857
CLASS I       311
CLASS IV      199
Name: facility_t, dtype: int64 

greenwave NO     5115
YES      48
No       12
Name: greenwave, dtype: int64 

date_last_ 2017-10-18    4595
2017-12-27     244
2017-12-11      70
2018-01-17      50
2017-12-28      42
2018-01-09      39
2018-07-20      27
2018-03-19      25
2018-05-08      16
2017-10-25      16
2018-08-14      13
2017-12-21      12
2018-07-13       7
2018-04-18       6
2018-08-13       5
2018-04-25       4
2018-05-25       2
2018-06-22       1
2018-04-24       1
Name: date_last_, dtype: int64 

last_edite JENWONG    5175
Name: last_edite, dtype: int64 

raised NO  

In [6]:
bikeway_map3['streetname'] = bikeway_map3['streetname'].astype('str')

In [11]:
traffic_reports_and_bikelanes = traffic_reports_bikelaneobjectids.merge( bikeway_map3, how= 'left', \
                                                                                 on = 'globalid', indicator = True, suffixes=('_accid', '_bikelane'))

In [12]:
traffic_reports_and_bikelanes["_merge"].value_counts()

left_only     1191
both           963
right_only       0
Name: _merge, dtype: int64

We have ~1200 traffic accidents that did not occur on or near a bike lane that existed in 2010. We should now figure out whether the accidents that occured near a bike lane in 2010 occured before or after the installation date.

In [13]:
# convert date of accident to datetime object. Compare bike lane installation date to it
traffic_reports_and_bikelanes['Date'] = pd.to_datetime(traffic_reports_and_bikelanes['Date'])
traffic_reports_and_bikelanes.loc[ traffic_reports_and_bikelanes['install_mo'].isnull(), 'install_mo'] = 1
traffic_reports_and_bikelanes.loc[ traffic_reports_and_bikelanes['install_yr_bikelane'].isnull(), 'install_yr_bikelane'] = 3000 #the year when ppl live underwater
traffic_reports_and_bikelanes['install_mo'] = traffic_reports_and_bikelanes['install_mo'].apply(int)
traffic_reports_and_bikelanes['install_yr_bikelane'] = traffic_reports_and_bikelanes['install_yr_bikelane'].apply(int)
traffic_reports_and_bikelanes['bikelane_installdate'] = traffic_reports_and_bikelanes['install_mo'].map(str) + '/01/' + traffic_reports_and_bikelanes['install_yr_bikelane'].map(str)
traffic_reports_and_bikelanes['bikelane_installdate'] =  pd.to_datetime(traffic_reports_and_bikelanes['bikelane_installdate'], errors='coerce')
traffic_reports_and_bikelanes['bikelane_exist'] = traffic_reports_and_bikelanes['bikelane_installdate'] < traffic_reports_and_bikelanes['Date']
traffic_reports_and_bikelanes['accident_year'] = traffic_reports_and_bikelanes['Date'].apply( lambda x: x.year)

In [14]:
traffic_reports_and_bikelanes['bikelane_exist'].value_counts()

False    1379
True      775
Name: bikelane_exist, dtype: int64

~1400 accidents that were not on an actual bike lane. 700 that were. A regression, especially one parsed by other arguements (such as year, neighborhood, day of week) would probably have issues with sample size. But I'll run one anyways after investigating the other variables for fun

Might be useful to aggregate by weekday vs weekend

In [15]:
def isWeekday( x ):
    if x in ( 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday' ):
        return('Weekday')
    else: # assuming no errors in our dataset, oc
        return('Weekend')

In [16]:
traffic_reports_and_bikelanes['isWeekday'] = traffic_reports_and_bikelanes.DayOfWeek.apply( lambda x: isWeekday( x))

In [17]:
traffic_reports_and_bikelanes.isWeekday.value_counts()

Weekday    1554
Weekend     600
Name: isWeekday, dtype: int64

Most accidents occur on a weekday! 

Let's check if some of the bike lanes variables are relevant in understanding where accidents occur. The variables include bike lane barrier type, bike lane type, whether or not a buffer exists

In [18]:
traffic_reports_and_bikelanes_onlyexist = traffic_reports_and_bikelanes.loc[traffic_reports_and_bikelanes['bikelane_exist'] == True ].reset_index(drop = True)

In [22]:
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'buffered'].value_counts(), "\n")
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'barrier'].value_counts(), "\n")
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'greenwave'].value_counts(), "\n")
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'sharrow'].value_counts(), "\n")
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'contraflow'].value_counts(), "\n")
print(traffic_reports_and_bikelanes_onlyexist.loc[traffic_reports_and_bikelanes_onlyexist['bikelane_exist'] == True, 'symbology'].value_counts(), "\n")

NO     738
YES     37
Name: buffered, dtype: int64 

SAFE-HIT POSTS                       26
PARKING; SAFE-HIT POSTS               8
CONCRETE; PARKING; SAFE-HIT POSTS     7
CONCRETE                              5
PARKING                               4
CONCRETE; SAFE-HIT POSTS              3
K-RAIL                                2
Name: barrier, dtype: int64 

NO     741
YES     34
Name: greenwave, dtype: int64 

0.0    486
1.0    284
5.0      2
4.0      2
2.0      1
Name: sharrow, dtype: int64 

NO    775
Name: contraflow, dtype: int64 

BIKE ROUTE           347
BIKE LANE            337
SEPARATED BIKEWAY     53
BIKE PATH             38
Name: symbology, dtype: int64 



I want to guage if there are duplicates in this database. I know from SQL queries on mode that there can be 1-2 bike lanes on roads that are both one and two way

It'd be interesting to look at accidents vs bike lane types, and if a sharrow exists since there seems to be a reasonable amount of accidents with sharrows, bike lanes. However, not enough of the other types (neighborways, contraflow routes)

## Road map of SF
I'm now interested in figuring out how much of SF contains bike lanes? This will help us understand why so many accidents didn't match to a bike lane.

In [19]:
#total_map = gpd.read_file('Streets - Active and Retired/geo_export_ea4de75d-ee93-4ded-8f06-09caff29c4e1.shp')
total_map = gpd.read_file("https://data.sfgov.org/resource/3psu-pn9h.geojson")

#total_map = pd.read_file("https://data.sfgov.org/resource/3psu-pn9h.geojson")

I know from [here](https://data.sfgov.org/Geographic-Locations-and-Boundaries/Streets-Active-and-Retired/3psu-pn9h) that the column ' tells us which roads are available in 2018. 

We want to get the length of road since this dataset doesn't have a length of lane (unlike the bikeway map). We use the haversine formula since that can take coordinates and find the distance between them, accounting for the curvature of the earth

In [21]:
from math import radians, cos, sin, asin, sqrt
# Calculates distance between 2 GPS coordinates 
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3959 # Radius of earth in kilometers. Use 3959 for miles
    return c * r

In [22]:
bikeway_map3.columns

Index(['barrier', 'biap', 'buffered', 'cnn', 'contraflow', 'date_creat',
       'time_creat', 'created_us', 'dir', 'direct', 'double', 'facility_t',
       'from_st', 'fy', 'globalid', 'greenwave', 'install_mo', 'install_yr',
       'date_last_', 'time_last_', 'last_edite', 'length', 'notes', 'number',
       'objectid', 'qtr', 'raised', 'shape_len', 'sharrow', 'sm_sweeper',
       'street', 'streetname', 'surface_tr', 'symbology', 'to_st', 'update_mo',
       'update_yr', 'censusbloc', 'geometry'],
      dtype='object')

So do we standardize our metric of accidents per length by area or by both sides of street? Let's get the lengths of each bike lane, the lengths of each road, and then join both to the list of accidents. we can then count the list of traffic accidents on bike lanes and on road maps through group bys? we could group by a neighborhood, and bike or road id vs road/bike lengths, and then sum the road or bike lengths and then count the accidents

what about time? group by time, day, bike/road id vs length, and then sum road/bike lengths, and then count the accidents. I just want to verify that we should do all this at the accident level and I stand by that

In [23]:
# calculate length of bike lanes
bikelane_length = []

for line in bikeway_map3.geometry:
    numCoords = len(line.coords) - 1
    distance = 0
    for i in range(0, numCoords):
        point1 = line.coords[i]
        point2 = line.coords[i + 1]
        distance += haversine(point1[0], point1[1], point2[0], point2[1])

    bikelane_length.append(distance)

In [25]:
bikeway_map3 = pd.concat( [bikeway_map3.reset_index(drop = True), pd.DataFrame( {'bikelane_length_haver': bikelane_length})], axis = 1)

In [27]:
# calculate length of road in miles
lengths_list = []

for i in range(0, len(total_map.geometry)):
    
    current_path = total_map.loc[i, 'geometry']
    
    numCoords = len(current_path.coords) - 1
    distance = 0
    for i in range(0, numCoords):
        point1 = current_path.coords[i]
        point2 = current_path.coords[i + 1]
        distance += haversine(point1[0], point1[1], point2[0], point2[1])
        
    # doubling the distance of 2 way bike lanes - accidents can happen at either side of the street
    if total_map.loc[i, 'oneway'] == 'B':
        distance *= 2

    lengths_list.append(distance)

In [34]:
total_map = pd.concat( [pd.DataFrame({'road_length': lengths_list}), total_map], axis = 1)

In [36]:
print((round(np.sum(bikeway_map3.length) / np.sum(lengths_list),4)*100), "%", "of roads have bikelanes. some of those roads may be inactive")

5.680000000000001 % of roads have bikelanes


total_map.nhood.value_counts() -> i would like higher level neighborhood groupings, but we can worry about that with the traffic accident police district dataset!

In [31]:
#how to check whether date_drop means that all lanes are inacive
print(set(total_map.loc[~total_map.date_dropped.isnull(), 'active']))
print(set(total_map.loc[total_map.date_dropped.isnull(), 'active']))

{False}
{True}


In [29]:
total_map3 = total_map

total_map3['date_dropped'] =  pd.to_datetime(total_map3.loc[~total_map3.date_dropped.isnull(), 'date_dropped']).apply( lambda x: pd.Timestamp(x))
total_map3['date_added'] =  pd.to_datetime(total_map3.loc[~total_map3.date_dropped.isnull(), 'date_added']).apply( lambda x: pd.Timestamp(x))

total_map3.loc[ total_map3.date_dropped.isnull(), 'date_added']  = pd.Timestamp(  dt.date(1900, 12, 31))
total_map3.loc[ total_map3.date_dropped.isnull(), 'date_dropped']  = pd.Timestamp(  dt.date(2021, 12, 31))

total_map3.geometry = gpd.GeoSeries(total_map3.geometry)
total_map3.crs = bikeway_map3.crs

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  # Remove the CWD from sys.path while we load stuff.


In [30]:
traffic_reports_and_bikelanes.Date = traffic_reports_and_bikelanes.Date.apply(lambda x: pd.Timestamp( x ))

In [31]:
traffic_reports_and_bikelanes2 = traffic_reports_and_bikelanes.drop_duplicates(subset=['X', 'Y', 'IncidntNum'], keep = 'first').reset_index( drop = True)

In [32]:
traffic_reports_and_bikelanes2['cnn_of_road'] = ['none']* traffic_reports_and_bikelanes2.shape[0]

In [55]:
np.isnan(traffic_reports_and_bikelanes2.loc[i, 'geometry'])

True

In [67]:
# returns closest road to a road i. 
# i don't rellly care if i don't get the most accurate road at this point, just the min is fine even if it's first

def geo_or_na( possible_geo ):
    if np.isnan(possible_geo).all():
        return None
    else:
        return traffic_reports_and_bikelanes2.loc[i, 'geometry'].distance(possible_geo)

for i in range(0, len(traffic_reports_and_bikelanes2.geometry) ):
    if np.isnan(traffic_reports_and_bikelanes2.loc[i, 'geometry']).all():
        traffic_reports_and_bikelanes2.loc[i,'cnn_of_road'] = None
        continue

    total_map_for_testing =  total_map3.loc[ (traffic_reports_and_bikelanes2.loc[i,'Date'] >= total_map3.date_added) & (total_map3.date_dropped <= traffic_reports_and_bikelanes2.loc[i,'Date'])  ,   ]
    distance_to_road = total_map_for_testing.geometry.apply( lambda x: geo_or_na(x) )
    
    if len(distance_to_road) == 0:
        traffic_reports_and_bikelanes2.loc[i,'cnn_of_road']  = None
    else:
        # uses row index of road that has min distance to bike lane in question to return cnn of the road with min distance
        traffic_reports_and_bikelanes2.loc[i,'cnn_of_road'] = total_map_for_testing.loc[distance_to_road.idxmin(), 'cnn'] 

In [69]:
# merge each lane with a bike lane

traffic_reports_and_bikelanes3 = traffic_reports_and_bikelanes2.merge(total_map3, how = 'left', left_on ='cnn_of_road', right_on = 'cnn', suffixes=('_accid', '_road'))

In [70]:
traffic_reports_and_bikelanes3 = traffic_reports_and_bikelanes3.merge( bikeway_map3[['globalid', 'bikelane_length_haver']] , how = 'left', on = 'globalid')

In [83]:
traffic_reports_and_bikelanes3['Time'] = pd.to_datetime(traffic_reports_and_bikelanes3['Time'])
traffic_reports_and_bikelanes3['hour'] = traffic_reports_and_bikelanes3['Time'].apply(lambda x: x.hour)

# Understanding Data - Sample Sizes of Different Groups

Bike lanes may or may not cause traffic accidents. In an ideal world where it's easy to understand the causal impact of something, the best way to analyze this would be to have two completely identical cities, that are basically the same except they're not, and randomly assign bike lanes to one and compare traffic accident numbers after a few weeks/years. That isn't possible.

Another way is to look at how number of accidents increased or decrease per bike lane, for the year's after it's installed. Let's see if we have enough variability in a regression to do that. For each bike lane, are there accidents before it existed and after it existed?

In [179]:
traffic_reports_and_bikelanes.bikelane_exist.value_counts()

False    1379
True      775
Name: bikelane_exist, dtype: int64

In [180]:
traffic_reports_and_bikelanes['_merge'].value_counts()

left_only     1191
both           963
right_only       0
Name: _merge, dtype: int64

Most accidents don't have bike lanes associated with it!

In [181]:
test = traffic_reports_and_bikelanes.groupby(['globalid', 'bikelane_exist']).count()['Date'].reset_index(drop = False)

Checking if a bike lane has both a before and after - if sum of both_true_false_check per globalid == 3, then they do

In [184]:
test['both_true_false_check'] = traffic_reports_and_bikelanes.bikelane_exist.apply(lambda x: 1 if x == True else 2)

In [188]:
sum(test.groupby(['globalid']).sum()['both_true_false_check'] == 3)

19

Only 19 bike lanes have accidents before and after! That is pretty disappointing and frankly wouldn't make a very good difference in difference at the bike lane level. I can run a regression of number of accidents on, and off a bike lane, for different neighorhoods per year, just to increase my sample size. 

But this regression would not be causal and I wouldn't get the results I want. I think it's better to investigate this problem through other means

There are not that many counts per year, and we don't have many years of data! This is something to note when 

In [411]:
traffic_reports_and_bikelanes.groupby(['accident_year', 'bikelane_exist']).count()['Date']

accident_year  bikelane_exist
2003           False              57
               True               10
2004           False              44
               True                8
2005           False              45
               True               12
2006           False              58
               True               23
2007           False              66
               True               29
2008           False              74
               True               25
2009           False              80
               True               33
2010           False              88
               True               63
2011           False              85
               True               57
2012           False             104
               True               67
2013           False              98
               True               49
2014           False             107
               True               59
2015           False              99
               True               79
2016    

Our options now are
* Higher level investigation of results -> I really want to examine if accidents are correlated with not off road because of lack of bike lanes or if standardizing number of accidents by length will help
* Visuals!